In [1]:
import torch
import torch.nn as nn
import torchvision.models as model


class VGG16(nn.Module):
    
    def __init__(self, num_classes, pretrained=False):
        super().__init__()
                
        self.vgg16 = model.vgg16(pretrained=pretrained)
        self.vgg16.classifier[-1] = torch.nn.Linear(in_features=4096, out_features=num_classes, bias=True)

    def forward(self, image):
        out = self.vgg16(image)
        
        return out


In [2]:
import torch
import torch.nn as nn
from torch.nn.functional import softmax
from torch.nn.functional import cross_entropy
from torchmetrics import F1Score
from torchvision import transforms

from torch.utils.data import DataLoader, Dataset
from torchvision.models import resnet18
from torchvision.models import vgg16
from datasets.dataset_retrieval import custom_dataset
from torch.optim import SGD, Adam
from torch.utils.tensorboard import SummaryWriter
import tqdm
from torchvision.datasets import ImageFolder
import os

save_model_path = "checkpoints/"


def val(model, data_val, loss_function, writer, epoch, device):
    f1score = 0
    f1 = F1Score(num_classes=47, task='multiclass')
    data_iterator = enumerate(data_val)  # Take batches
    f1_list = []
    f1t_list = []

    with torch.no_grad():
        model.eval()  # Switch model to evaluation mode
        tq = tqdm.tqdm(total=len(data_val))
        tq.set_description('Validation:')

        total_loss = 0

        for _, batch in data_iterator:
            # Forward propagation
            image, label = batch
            image = image.to(device)
            label = label.to(device).long()  # Convert labels to LongTensor
            
            pred = model(image)

            loss = loss_function(pred, label)

            pred = pred.softmax(dim=1)
            
            f1_list.extend(torch.argmax(pred, dim=1).tolist())
            f1t_list.extend(label.tolist())  # Labels are already in LongTensor format

            total_loss += loss.item()
            tq.update(1)

    f1score = f1(torch.tensor(f1_list), torch.tensor(f1t_list))
    writer.add_scalar("Validation F1", f1score, epoch)
    writer.add_scalar("Validation Loss", total_loss / len(data_val), epoch)

    tq.close()
    print("F1 score: ", f1score)

    return None


def train(model, train_loader, val_loader, optimizer, loss_fn, n_epochs, device, log_dir):
    # Scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=4, gamma=0.1)
    writer = SummaryWriter(log_dir=os.path.join('runs', log_dir))

    model.to(device)  # Move the model to the specified device (e.g., GPU or CPU)
    model.train()  # Set the model to training mode

    for epoch in range(n_epochs):
        model.train()
        running_loss = 0.0

        tq = tqdm.tqdm(total=len(train_loader))
        tq.set_description('epoch %d' % epoch)

        for i, (images, labels) in enumerate(train_loader):
            images = images.to(device)  # Move the batch of images to the specified device
            labels = labels.to(device).long()  # Convert labels to LongTensor
            
            optimizer.zero_grad()  # Reset the gradients of the optimizer

            # Forward pass
            outputs = model(images)

            # Compute loss
            loss = loss_fn(outputs, labels)
            outputs = outputs.softmax(dim=1)

            # Backward pass
            loss.backward()

            # Update model parameters
            optimizer.step()
            # Scheduler.step()

            running_loss += loss.item()
            tq.set_postfix(loss_st='%.6f' % loss.item())
            tq.update(1)

        writer.add_scalar("Training Loss", running_loss / len(train_loader), epoch)

        tq.close()
        epoch_loss = running_loss / len(train_loader)
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch + 1, n_epochs, epoch_loss))

        # Check the performance of the model on the validation dataset
        val(model, val_loader, loss_fn, writer, epoch, device)

        # Save the model in .pth format
        checkpoint = {
            'epoch': epoch + 1,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict()
        }

        torch.save(checkpoint, os.path.join(save_model_path, log_dir + '.pth'))
        print("Saved the model to " + save_model_path)




# def main(model_name, optimizer_name, lr, pretrained, log_dir):
    
#     device = "cuda"
#     torch.cuda.set_per_process_memory_fraction(0.8, device=0)  # Limit GPU memory to 80%

    
#     tr_train = transforms.Compose([
#         transforms.RandomHorizontalFlip(0.5),
#         transforms.RandomRotation(30),
#         # transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1),
#         transforms.Resize([250, 250]),
#         transforms.RandomCrop(224),
#         transforms.ToTensor(),

#     ])
    
#     tr_val = transforms.Compose([
#         transforms.Resize([224, 224]),
#         transforms.ToTensor(),

#     ])

#     train_data = ImageFolder(root=r"datasets\split_data\train", transform=tr_train)
#     val_data = ImageFolder(root=r"datasets\split_data\val", transform=tr_val)

#     train_loader = DataLoader(
#         train_data,
#         batch_size=8,
#         shuffle=True,

#     )

#     val_loader = DataLoader(
#         val_data,
#         batch_size=8,
#         drop_last=True,

#     )
    
#     max_epoch = 20

#     if model_name=='resnet':
#         model = resnet18(num_classes=4, pretrained=pretrained).to(device)
#     elif model_name=='vgg':
#         model = VGG16(num_classes=4, pretrained=pretrained).to(device)
    
#     if optimizer_name=='sgd':
#         optimizer = SGD(model.parameters(), lr=lr, momentum = 0.9)
#     elif optimizer_name=='adam':
#         optimizer = Adam(model.parameters(), lr=lr)
    
#     loss = nn.CrossEntropyLoss()

#     train(model, train_loader, val_loader, optimizer, loss, max_epoch, device, log_dir)
    
    
# if __name__ == "__main__":

    
#     # main(model_name='vgg', optimizer_name='sgd', lr=1e-3, pretrained=False, log_dir='vgg_sgd')
#     # main(model_name='vgg', optimizer_name='adam', lr=1e-4, pretrained=False, log_dir='vgg_adam')
    
#     main(model_name='vgg', optimizer_name='sgd', lr=1e-3, pretrained=True, log_dir='vgg_sgd_pretrained')
#     main(model_name='vgg', optimizer_name='adam', lr=1e-4, pretrained=True, log_dir='vgg_adam_pretrained')

In [ ]:
# Test function
def test(model, test_loader, device):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)  # Get the predicted class
            total += labels.size(0)  # Total number of labels
            correct += (predicted == labels).sum().item()  # Correct predictions

    accuracy = 100 * correct / total  # Accuracy in percentage
    print(f'Accuracy on the test set: {accuracy:.2f}%')
    return accuracy

In [6]:
import torch
import torch.nn as nn
import torchvision.models as model
from torchmetrics import F1Score
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import os
import tqdm

# Define the VGG16 model class
class VGG16(nn.Module):
    def __init__(self, num_classes, pretrained=False):
        super().__init__()
        self.vgg16 = model.vgg16(pretrained=pretrained)
        self.vgg16.classifier[-1] = torch.nn.Linear(in_features=4096, out_features=num_classes, bias=True)

    def forward(self, image):
        out = self.vgg16(image)
        return out

# Function to evaluate accuracy on the test dataset
def test(model, test_loader, device):
    correct_preds = 0
    total_preds = 0
    model.eval()  # Switch model to evaluation mode

    with torch.no_grad():
        for images, labels in tqdm.tqdm(test_loader, desc="Testing"):
            images = images.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)  # Get the predicted class

            # Update counts
            correct_preds += (predicted == labels).sum().item()
            total_preds += labels.size(0)

    accuracy = correct_preds / total_preds
    print(f"Test Accuracy: {accuracy * 100:.2f}%")
    return accuracy

# Function to load model from checkpoint
def load_model(checkpoint_path, model, device):
    checkpoint = torch.load(checkpoint_path, map_location=device)
    model.load_state_dict(checkpoint['state_dict'])
    model.to(device)
    model.eval()
    print(f"Model loaded successfully from {checkpoint_path}")
    return model

# Function to prepare the test dataset
def prepare_test_data(test_data_path, batch_size):
    tr_test = transforms.Compose([
        transforms.Resize([224, 224]),  # Resize images
        transforms.ToTensor(),  # Convert to tensor
    ])
    
    # Load the test dataset
    test_data = ImageFolder(root=test_data_path, transform=tr_test)
    test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)  # No need to shuffle for testing
    print(f"Test dataset size: {len(test_data)} samples")
    return test_loader

def main(model_name, optimizer_name, lr, pretrained, log_dir, test_data_path):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    torch.cuda.set_per_process_memory_fraction(0.8, device=0)  # Limit GPU memory to 80%

    if model_name == 'vgg':
        model = VGG16(num_classes=4, pretrained=pretrained).to(device)

    # Load the model
    checkpoint_path = os.path.join("checkpoints", log_dir + '.pth')
    model = load_model(checkpoint_path, model, device)

    # Prepare the test data
    test_loader = prepare_test_data(test_data_path, batch_size=8)

    # Evaluate the model on the test data
    test_accuracy = test(model, test_loader, device)
    print(f"Test Accuracy for {model_name} ({log_dir}): {test_accuracy:.4f}")

if __name__ == "__main__":
    test_data_path = r"datasets\split_data\test"  # Path to your test dataset

    # Example: Test with a pretrained VGG model trained with SGD
    main(model_name='vgg', optimizer_name='sgd', lr=1e-3, pretrained=True, log_dir='vgg_sgd_pretrained', test_data_path=test_data_path)
    main(model_name='vgg', optimizer_name='sgd', lr=1e-3, pretrained=True, log_dir='vgg_adam_pretrained', test_data_path=test_data_path)


c:\Users\ASUS\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\ASUS\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_19348\408221922.py:47: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See 

Model loaded successfully from checkpoints\vgg_sgd_pretrained.pth
Test dataset size: 804 samples


Testing: 100%|██████████| 101/101 [00:04<00:00, 21.49it/s]


Test Accuracy: 84.20%
Test Accuracy for vgg (vgg_sgd_pretrained): 0.8420
Model loaded successfully from checkpoints\vgg_adam_pretrained.pth
Test dataset size: 804 samples


Testing: 100%|██████████| 101/101 [00:04<00:00, 20.33it/s]

Test Accuracy: 79.73%
Test Accuracy for vgg (vgg_adam_pretrained): 0.7973


In [3]:
def main(model_name, optimizer_name, lr, pretrained, log_dir):
    
    device = "cuda"
    torch.cuda.set_per_process_memory_fraction(0.8, device=0)  # Limit GPU memory to 80%

    
    tr_train = transforms.Compose([
        transforms.RandomHorizontalFlip(0.5),
        transforms.RandomRotation(30),
        # transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1),
        transforms.Resize([250, 250]),
        transforms.RandomCrop(224),
        transforms.ToTensor(),

    ])
    
    tr_val = transforms.Compose([
        transforms.Resize([224, 224]),
        transforms.ToTensor(),

    ])

    train_data = ImageFolder(root=r"datasets\split_data\train", transform=tr_train)
    val_data = ImageFolder(root=r"datasets\split_data\val", transform=tr_val)

    train_loader = DataLoader(
        train_data,
        batch_size=8,
        shuffle=True,

    )

    val_loader = DataLoader(
        val_data,
        batch_size=8,
        drop_last=True,

    )
    
    max_epoch = 20

    if model_name=='resnet':
        model = resnet18(num_classes=4, pretrained=pretrained).to(device)
    elif model_name=='vgg':
        model = VGG16(num_classes=4, pretrained=pretrained).to(device)
    
    if optimizer_name=='sgd':
        optimizer = SGD(model.parameters(), lr=lr, momentum = 0.9)
    elif optimizer_name=='adam':
        optimizer = Adam(model.parameters(), lr=lr)
    
    loss = nn.CrossEntropyLoss()

    train(model, train_loader, val_loader, optimizer, loss, max_epoch, device, log_dir)
    
    
if __name__ == "__main__":

    
    main(model_name='vgg', optimizer_name='sgd', lr=0.0005, pretrained=False, log_dir='vgg_sgd')
    main(model_name='vgg', optimizer_name='adam', lr=0.00005, pretrained=False, log_dir='vgg_adam')

c:\Users\ASUS\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\ASUS\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
epoch 0: 100%|██████████| 350/350 [00:55<00:00,  6.36it/s, loss_st=1.386817]


Epoch [1/20], Loss: 1.3874


Validation:: 100%|██████████| 50/50 [00:04<00:00, 12.37it/s]


F1 score:  tensor(0.2875)
Saved the model to checkpoints/


epoch 1: 100%|██████████| 350/350 [00:40<00:00,  8.67it/s, loss_st=1.387679]


...

Epoch [17/20], Loss: 1.3866


Validation:: 100%|██████████| 50/50 [00:02<00:00, 20.23it/s]


F1 score:  tensor(0.2500)
Saved the model to checkpoints/


epoch 17:  22%|██▏       | 78/350 [00:13<00:45,  5.98it/s, loss_st=1.385767]

KeyboardInterrupt: 

In [4]:
import torch
import torch.nn as nn
import torchvision.models as model
from torchmetrics import F1Score
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import os
import tqdm

# Define the VGG16 model class
class VGG16(nn.Module):
    def __init__(self, num_classes, pretrained=False):
        super().__init__()
        self.vgg16 = model.vgg16(pretrained=pretrained)
        self.vgg16.classifier[-1] = torch.nn.Linear(in_features=4096, out_features=num_classes, bias=True)

    def forward(self, image):
        out = self.vgg16(image)
        return out

# Function to evaluate accuracy on the test dataset
def test(model, test_loader, device):
    correct_preds = 0
    total_preds = 0
    model.eval()  # Switch model to evaluation mode

    with torch.no_grad():
        for images, labels in tqdm.tqdm(test_loader, desc="Testing"):
            images = images.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)  # Get the predicted class

            # Update counts
            correct_preds += (predicted == labels).sum().item()
            total_preds += labels.size(0)

    accuracy = correct_preds / total_preds
    print(f"Test Accuracy: {accuracy * 100:.2f}%")
    return accuracy

# Function to load model from checkpoint
def load_model(checkpoint_path, model, device):
    checkpoint = torch.load(checkpoint_path, map_location=device)
    model.load_state_dict(checkpoint['state_dict'])
    model.to(device)
    model.eval()
    print(f"Model loaded successfully from {checkpoint_path}")
    return model

# Function to prepare the test dataset
def prepare_test_data(test_data_path, batch_size):
    tr_test = transforms.Compose([
        transforms.Resize([224, 224]),  # Resize images
        transforms.ToTensor(),  # Convert to tensor
    ])
    
    # Load the test dataset
    test_data = ImageFolder(root=test_data_path, transform=tr_test)
    test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)  # No need to shuffle for testing
    print(f"Test dataset size: {len(test_data)} samples")
    return test_loader

def main(model_name, optimizer_name, lr, pretrained, log_dir, test_data_path):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    torch.cuda.set_per_process_memory_fraction(0.8, device=0)  # Limit GPU memory to 80%

    if model_name == 'vgg':
        model = VGG16(num_classes=4, pretrained=pretrained).to(device)

    # Load the model
    checkpoint_path = os.path.join("checkpoints", log_dir + '.pth')
    model = load_model(checkpoint_path, model, device)

    # Prepare the test data
    test_loader = prepare_test_data(test_data_path, batch_size=8)

    # Evaluate the model on the test data
    test_accuracy = test(model, test_loader, device)
    print(f"Test Accuracy for {model_name} ({log_dir}): {test_accuracy:.4f}")

if __name__ == "__main__":
    test_data_path = r"datasets\split_data\test"  # Path to your test dataset

    # Example: Test with a pretrained VGG model trained with SGD
    main(model_name='vgg', optimizer_name='sgd', lr=1e-3, pretrained=True, log_dir='vgg_sgd', test_data_path=test_data_path)
    main(model_name='vgg', optimizer_name='sgd', lr=1e-3, pretrained=True, log_dir='vgg_adam', test_data_path=test_data_path)


c:\Users\ASUS\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_18680\3006699582.py:47: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbit

Model loaded successfully from checkpoints\vgg_sgd.pth
Test dataset size: 804 samples


Testing: 100%|██████████| 101/101 [00:08<00:00, 11.27it/s]


Test Accuracy: 29.35%
Test Accuracy for vgg (vgg_sgd): 0.2935
Model loaded successfully from checkpoints\vgg_adam.pth
Test dataset size: 804 samples


Testing: 100%|██████████| 101/101 [00:04<00:00, 21.05it/s]

Test Accuracy: 25.00%
Test Accuracy for vgg (vgg_adam): 0.2500


In [7]:
from collections import Counter
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

def main():
    device = "cuda"
    torch.cuda.set_per_process_memory_fraction(0.8, device=0)  # Limit GPU memory to 80%

    # Data transformations
    tr_train = transforms.Compose([
        transforms.RandomHorizontalFlip(0.5),
        transforms.RandomRotation(30),
        transforms.Resize([250, 250]),
        transforms.RandomCrop(224),
        transforms.ToTensor(),
    ])
    
    tr_val = transforms.Compose([
        transforms.Resize([224, 224]),
        transforms.ToTensor(),
    ])

    # Load train and validation datasets
    train_data = ImageFolder(root=r"datasets\split_data\train", transform=tr_train)
    val_data = ImageFolder(root=r"datasets\split_data\val", transform=tr_val)

    # Create DataLoaders
    train_loader = DataLoader(train_data, batch_size=8, shuffle=True)
    val_loader = DataLoader(val_data, batch_size=8, drop_last=True)

    # Analyze label distribution for training data
    train_labels = []
    for inputs, labels in train_loader:
        train_labels.extend(labels.tolist())
    train_label_counts = Counter(train_labels)

    print("\nLabel Distribution in Training Set:")
    for label, count in train_label_counts.items():
        print(f"Class {label}: {count} samples")

    # Analyze label distribution for validation data
    val_labels = []
    for inputs, labels in val_loader:
        val_labels.extend(labels.tolist())
    val_label_counts = Counter(val_labels)

    print("\nLabel Distribution in Validation (Test) Set:")
    for label, count in val_label_counts.items():
        print(f"Class {label}: {count} samples")

main()



Label Distribution in Training Set:
Class 3: 699 samples
Class 1: 699 samples
Class 2: 699 samples
Class 0: 699 samples

Label Distribution in Validation (Test) Set:
Class 0: 100 samples
Class 1: 100 samples
Class 2: 100 samples
Class 3: 100 samples
